In [ ]:
import pandas as pd
import numpy as np
import json
import plotly.express as px


## Exercise 1

First we load and take a look at the dataset

In [ ]:
dessert_data = pd.read_json("data/dessert.json")
dessert_data.drop(["portions", "nutrients"],axis=1,inplace=True)
dessert_data.sample(10)

### Exercise 1a)

We decided to replace empty Strings and `None` values with the string "No manufacturer" for the **Manufacturer** column:

In [ ]:
dessert_data.replace("", None, inplace=True)
dessert_data.fillna(value="no Manufacturer",inplace=True)
dessert_data["manufacturer"].sample(40)

Next, we need to extract the contents of the lists in the **tags** column. We define a function to print the content of the list and return it:

In [ ]:
def print_list(my_list):
    if len(my_list) > 0:
        count = 0
        for x in my_list:
            print(f"list value of element {count}: {x}")
            count += 1
    return my_list
dessert_data["tags"].map(print_list)

As you can see, we have only 4 list entries actually containing any tags in the entire column. And for those entries, the list has a length of 1, which means we can use a similiar function to extract the values from the list and return them as a string to the dataframe:

In [ ]:
def extract_list_ele(my_list): #returns first content of list if non empty and "no Tags" otherwise
    if type(my_list) == list: # check if we are working on lists
        if len(my_list) == 0:
            return "no Tags"
        else:
            return my_list[0]
    else: #if not list, do nothing
        return my_list

dessert_data["tags"]=dessert_data["tags"].map(extract_list_ele)

dessert_data['tags'].sample(10)

We need to consolidate different types of spellings in the `type` column:

In [ ]:
dessert_data["type"].unique()

In [ ]:
dessert_data.replace("Sweet Breads", "Sweet Bread", inplace=True)
dessert_data.replace("sweet bread", "Sweet Bread", inplace=True)
dessert_data.replace("Sweet bread", "Sweet Bread", inplace=True)

dessert_data.replace("Cookies", "Cookie", inplace=True)
dessert_data.replace("Coookies", "Cookie", inplace=True)

dessert_data.replace("Biscuits", "Biscuit", inplace=True)

dessert_data.replace("Muffins", "Muffin", inplace=True)

dessert_data.replace("Savory bread", "Savory Bread", inplace=True)

Furthermore, we are going to flatten the `nutrients` and `portions` columns with `json_normalize()`:

In [ ]:
with open("data/dessert.json") as json_file: 
    dessert_json = json.load(json_file)
    
dessert_data_port = pd.json_normalize(dessert_json, record_path=["portions"],meta=["id","type"],meta_prefix="origin-") # flatten json object into separate df
dessert_data_nutr = pd.json_normalize(dessert_json, record_path=["nutrients"],meta=["id","type"], meta_prefix="origin-") # flatten json object into separate df

since we are loading from the json, we need to consolidate `type` again:

In [ ]:
dessert_data_nutr.replace("Sweet Breads", "Sweet Bread", inplace=True)
dessert_data_nutr.replace("sweet bread", "Sweet Bread", inplace=True)
dessert_data_nutr.replace("Sweet bread", "Sweet Bread", inplace=True)

dessert_data_nutr.replace("Cookies", "Cookie", inplace=True)
dessert_data_nutr.replace("Coookies", "Cookie", inplace=True)

dessert_data_nutr.replace("Biscuits", "Biscuit", inplace=True)

dessert_data_nutr.replace("Muffins", "Muffin", inplace=True)

dessert_data_nutr.replace("Savory bread", "Savory Bread", inplace=True)



dessert_data_port.replace("Sweet Breads", "Sweet Bread", inplace=True)
dessert_data_port.replace("sweet bread", "Sweet Bread", inplace=True)
dessert_data_port.replace("Sweet bread", "Sweet Bread", inplace=True)

dessert_data_port.replace("Cookies", "Cookie", inplace=True)
dessert_data_port.replace("Coookies", "Cookie", inplace=True)

dessert_data_port.replace("Biscuits", "Biscuit", inplace=True)

dessert_data_port.replace("Muffins", "Muffin", inplace=True)

dessert_data_port.replace("Savory bread", "Savory Bread", inplace=True)

lets look at the nutr table and check it for none types:

In [ ]:
nutr_none =dessert_data_nutr[dessert_data_nutr["type"].isna()] #filter out all rows containing a None type val
nutr_none

We only have one single entry which is None, so we can overwrite it manually with the "Vitamin" type:

In [ ]:
dessert_data_nutr.at[8443,"type"] = "Vitamins"
nutr_none =dessert_data_nutr[dessert_data_nutr["type"].isna()]
nutr_none #should be empty now

Next, there is one negative value which we need to set to zero: 

In [ ]:
#dessert_data_nutr['value'].describe
dessert_data_nutr[dessert_data_nutr["value"]< 0]

In [ ]:
dessert_data_nutr.at[416, "value"] = 0
dessert_data_nutr[dessert_data_nutr["value"]<0]

### Exercise 1b)



In [ ]:
#get unique dessert types
dessert_data["type"].unique()


In [ ]:
#get dessert type distribution
dessert_data["type"].value_counts().plot.bar()

### Exercise 1 c)

We need to display the median of vitamins by food type. In order to achieve this, we use the `groupby()` method:

In [ ]:
nutr_vit = dessert_data_nutr.query('type.str.contains("Vitamins")', engine = 'python') # only keep entries with type Vitamins
nutr_vit.drop("origin-id",axis=1).groupby(["origin-type"]).median(True)

### Exercise 1 d)

We are tasked with detecting outliers in the amount of available vitamins with a some different methods. We are going to start with a box plot:

In [ ]:
fig = px.box(nutr_vit, x = "value",y = "type")
fig

A different approach to spot outliers in a dataset is with a scatter plot. It is quite similar to box plot, but it is usually easier to see outlier data with a scatter plot.

In [ ]:
# scatter plot
fig = px.scatter(nutr_vit, y='value', title='Outlier detection with a scatter plot', color='value', color_continuous_scale='thermal')
fig.show()

We can check the distribution of the data to see which statistical approach can be used.

In [ ]:
# draw a histogram to check value distribution
fig = px.histogram(nutr_vit['value'], x='value', title='Distribution of the values of Vitamins')
fig.show()

Since the data is not normally distirbuted, we can use the interquartile range (IQR) method instead of using the Z-score. Normally the IQR is used with the 25th and the 75th percentile of a given dataset. Our data seems very spread and so we decided to adjust the percentiles for lower and upper barrier individually.

In [ ]:
# find outliers with defined 'barriers' of min and max (in %) and then apply this as a filter on the given dataframe
def find_outliers_IQR(df, min, max):
   q1=df.quantile(min / 100)
   q3=df.quantile(max / 100)

   IQR=q3-q1

   outliers = df[((df<(q1-1.5*IQR)) | (df>(q3+1.5*IQR)))]

   return outliers

In [ ]:
# apply function to dataframe and store the outliers
outliers = find_outliers_IQR(nutr_vit['value'], 5, 95)

print("number of outliers: "+ str(len(outliers)))
print("max outlier value: "+ str(outliers.max()))
print("min outlier value: "+ str(outliers.min()))

outliers